# 🦙 Ollama

<center>
    <img src="https://ollama.com/public/cloud.png" alt="ollama is cool" width="200"/>
    <img src="https://mintcdn.com/ollama-9269c548/w-L7kuDqk3_8zi5c/images/welcome.png?w=840&fit=max&auto=format&n=w-L7kuDqk3_8zi5c&q=85&s=cc3770b69817332b5961d0faecdce473" alt="ollama working" width="400"/>
</center>



As we can see in these picture (^^), [Ollama](https://docs.ollama.com/) is a cool, simple and lightweight tool that allows LLMs to be run locally without complex configuration. 

### 🏳️ Open the cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


launch a new terminal session

<center>
    <img src="https://icon-library.com/images/bash-icon/bash-icon-6.jpg" alt="bash is also cool" width="200"/>
</center>


and execute therein the following commands to load the previously prepared enviroment with ollama installed

```bash
module load ollama
```

launch an ollama server

```bash
export OLLAMA_MODELS=/pbs/throng/training/universite-hiver/cache/ollama/models
export CUDA_VISIBLES_DEVICES=GPU-99ec877c-53b2-49ba-a1ea-0b6f0d0f0c08; 
export OLLAMA_HOST=http://127.0.0.1:11435; 
export OLLAMA_FLASH_ATTENTION=1; 
ollama serve &
```

and pull `llama3.1:8b` model

```bash
ollama pull llama3.1:8b
```

Check that `llama3.1:8b` model is present and show some informations

```bash
ollama list
ollama show llama3.1:8b
```

Create one request to ollama's server:

```bash
curl -X POST http://localhost:11435/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
  "model": "llama3.1:8b",
  "messages": [{ "role": "user", "content": "Who won the last presidential elections in France?" }]
}'
```

Keep that terminal open for the rest of the tutorial, we will check therein ollama server logs.

### 🏴 Open another cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


and launch now the Notebook called `Python 3.13 (Ollama)`.

We will work from now in that notebook.

In [1]:
! /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/xan headers /pbs/home/u/univ-hiver01/private/resinwinterschool/hashtag_donaldtrump_6k_joebiden_25k_shuffled.csv  

0   created_at
1   tweet_id
2   tweet
3   likes
4   retweet_count
5   source
6   user_id
7   user_name
8   user_screen_name
9   user_description
10  user_join_date
11  user_followers_count
12  user_location
13  lat
14  long
15  city
16  country
17  continent
18  state
19  state_code
20  collected_at


In [2]:
! /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/xan select tweet hashtag_donaldtrump_6k_joebiden_25k_shuffled.csv | /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/xan v 


Displaying 1 col from 100 first rows of <stdin>
┌────┬─────────────────────────────────────────────────────────────────────┐
│ -  │ tweet                                                               │
├────┼─────────────────────────────────────────────────────────────────────┤
│ 0  │ #HunterBidenEmails #HunterBiden #BidenCrimeFamily #Biden #BidenHar… │
│ 1  │ "It's just decency... I'm less concerned about me but the people [… │
│ 2  │ "Mostly peaceful" #campaign-finance violations?\n#CorruptDemocrats… │
│ 3  │ #Biden https://t.co/qMs0PmUev5                                      │
│ 4  │ #Biden -- #Kamala is bought &amp; bossed. Just a raccoon &amp; tok… │
│ 5  │ @Liberal_Lady5 #Socialism is for losers! \n\n#JoeBiden is for lose… │
│ 6  │ Painted myself a big yard sign! #VoteBidenHarris2020 \n#Vote \n#Jo… │
│ 7  │ i'm taking the night off from watching ...\n\n@NBCUniversal and @M… │
│ 8  │ @realDonaldTrump Vote #BIDEN/HARRIS                                 │
│ 9  │ #BigGuy #JoeBiden\n#

##### Import python packages

In [3]:
import csv
import time
import asyncio
from openai import OpenAI
from string import Template
import nest_asyncio
nest_asyncio.apply()

##### Load inputs and create asynchronous iterator

In [4]:
input_file = "/pbs/home/u/univ-hiver01/private/resinwinterschool/hashtag_donaldtrump_6k_joebiden_25k_shuffled.csv"
content_column = 'tweet'

start = time.time()
with open(input_file, 'r') as f:
    tweets = [d[content_column] for d in csv.DictReader(f)]
end = time.time()
print(f"Loading {len(tweets)} tweets tooks {end - start} seconds.")


Loading 31000 tweets tooks 0.1941382884979248 seconds.


##### Load choices and extra body dictionary with structured_outputs options

In [8]:
choices_file = "multiple_us.txt"
with open(choices_file, 'r') as f:
    choice = f.read().split("\n")[:-1]

extra_body = {"structured_outputs": {"choice": choice}}

print(extra_body)

{'structured_outputs': {'choice': ['Biden', 'Trump', 'None']}}


##### Load instructions

In [9]:
instructions_file = "user_prompt_voteintention_multiple_all_english_us.txt"
with open(instructions_file, 'r') as f:
    instructions = f.read()
print(instructions)

Please classify the following social media message (posted in the weeks leading up to the 2020 US presidential election) according to whether it explicitly expresses the intention  of the author to vote for or calls for a vote for any of the candidates in that election: Donald Trump or Joy Biden, or whether it expresses neither of these intentions. Your answer should be based solely on the information contained in the message. Do not confuse a simple mention of a candidate with an intention or call to vote for him. Do not assume that a message containing only positive opinions about a particular candidate explicitly expresses the intention to vote for that candidate. Do not assume that a message corresponding to the opinions or political positions of a particular candidate necessarily expresses the intention to vote for that candidate. Do not confuse retweets, indirect speech or a quote to another person with the opinion of the author of the message. Be concise and respond only with th

##### Create ollama openai client

In [10]:
client = OpenAI(
    base_url="http://127.0.0.1:11435/v1/", # this is the same HOST given at ollama server launching
    api_key="EMPTY"  # required but unused
)

##### Create async functions to asynchronously request llm server

`asyncio` is a library to write concurrent code using the async/await syntax.

`asyncio` is used as a foundation for multiple Python asynchronous frameworks that provide high-performance network and web-servers, database connection libraries, distributed task queues, etc.



<center>
    <img src="https://cdn.prod.website-files.com/5d2dd7e1b4a76d8b803ac1aa/5fc4d4d7dea01e43ee075dbf_B6oeReCTEE90L7UN3fOOyhgqQO_hY4OZvFUTmXcPMjYIByNrdkYaXRe0qaAxgz9BlenAkd4_hgcw5OXKk8mTLjn1u-BeqEoEtSbW-SPwLzCZ2yeW-3gPJLOoMbc910y6IvaVziHG.png" alt="bash is also cool" width="700"/>
</center>

In [11]:
async def inputsIterator(start, end):
    for tweet in tweets[start:end]:
        yield tweet

In [12]:
async def doCompletetion(tweet):
    return client.chat.completions.create(
        model=client.models.list().data[0].id,
        messages=[{
            'role': 'user',
            'content': Template(instructions).substitute(tweet=tweet)
        }],
        extra_body=extra_body,
        max_completion_tokens=16,
        temperature=0.7,
        top_p=0.95,
    )

In [13]:
async def run_all(start, end):
    # Asynchronously call the function for each prompt
    tasks = [
        doCompletetion(tweet)
        async for tweet in inputsIterator(start, end)
    ]
    # Gather and run the tasks concurrently
    results = await asyncio.gather(*tasks)
    return results

In [29]:
# Run all courutines
init_idx = 1111
nb_tweets = 1

start = time.time()
results = asyncio.run(run_all(start=init_idx, end=init_idx + nb_tweets))
elapsed = time.time() - start

whole_db_nb_tweets = int(1e6)


print(f"""
Annotationg {nb_tweets} tweets took {elapsed} seconds,
this is {elapsed / nb_tweets} seconds per tweet or
{(1 / (24 * 3600)) * whole_db_nb_tweets * elapsed / nb_tweets} days for the whole database of {whole_db_nb_tweets} inputs.
""")


Annotationg 1 tweets took 1.2059476375579834 seconds,
this is 1.2059476375579834 seconds per tweet or
13.957727286550732 days for the whole database of 1000000 inputs.



> Annotationg 500 tweets took 310.1631088256836 seconds,
>
> this is 0.6203262176513672 seconds per tweet or
>
> 7.494696641185019 days for the whole database of 1043873 inputs.

In [22]:
for tweet, answer in zip(tweets, [r.choices[0].message.content for r in results]):
    if answer == "None":
        print("-----------------------------------------------" + "\n" + f"tweet: {tweet}" + "\n" + f"answer: {answer}")

-----------------------------------------------
tweet: #HunterBidenEmails #HunterBiden #BidenCrimeFamily #Biden #BidenHarris2020 #TwitterCensorship #censorship #FacebookCensorship https://t.co/2sD4rovM1f
answer: None
-----------------------------------------------
tweet: "It's just decency... I'm less concerned about me but the people [around me]."

THANK YOU for saying it for the people who need to learn empathy!! 
#biden #townhall #BidenTownHall #TownHalls https://t.co/lLV8GvPEUZ
answer: None
-----------------------------------------------
tweet: "Mostly peaceful" #campaign-finance violations?
#CorruptDemocrats #HunterBidenEmails #JoeBiden #ukraine #VOTE #Election2020 https://t.co/wfR6jjV6Mv
answer: None
-----------------------------------------------
tweet: #Biden -- #Kamala is bought &amp; bossed. Just a raccoon &amp; token. Biden's been complicit in poisoning water, bombing other nations, &amp; deporting at alarming rates while giving corporations handouts. https://t.co/2ZLcs38kAs